<a href="https://colab.research.google.com/github/DeMaagdJ/TurTRade/blob/main/TurtTRade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage
import pandas as pd
import math
import numpy as np
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from datetime import date, datetime, timedelta
import warnings
warnings.filterwarnings("ignore")





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Financial reviews/Robinhood.xlsx"
sheet_name = "Log"
rh_log = pd.read_excel(path, sheet_name)

rh_log['Date'] = rh_log["Date"].astype(str)


#Datatype is in datetime.date.  Datatype in the dataframe is in datetime64.
date55 = (date.today() - timedelta(days=55)).strftime("%Y-%m-%d")
date20 = (date.today() - timedelta(days=20)).strftime("%Y-%m-%d")

d1 = rh_log.loc[rh_log["Date"] == date55]
d2 = rh_log.loc[rh_log["Date"] == date20]


rows_less_than_zero = []

for index, row in d1.iterrows():
  if row['Cash Value'] < 0:
    rows_less_than_zero.append(row)
for index, row in d2.iterrows():
  if row['Cash Value'] < 0:
    rows_less_than_zero.append(row)

d_fin = pd.DataFrame(rows_less_than_zero)

display(d_fin)


In [ ]:
# Set your Alpha Vantage API key
api_key = "AlphaVanAPIkey"

# Create an instance of the TimeSeries & ATR indicator class with your API key
ts = TimeSeries(key=api_key)
#ti = TechIndicators(key= api_key)

symbols = []

for sym in range(1,5):
    sym = str(input("please enter a symbol "))
    symbols.append(sym)

# Create an empty DataFrame to store the data
combined_data = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Symbol'])

# Fetch the stock data for each symbol and append to the DataFrame
for symbol in symbols:
    data_ts, meta_data = ts.get_daily(symbol, outputsize='compact')
    #data_ti, meta_data = ti.get_atr(symbol, interval='daily', time_period=1)

          # Append symbol to each data point


    symbol_data = []
    for date, data_ts in data_ts.items():
        open_price = float(data_ts['1. open'])
        high_price = float(data_ts['2. high'])
        low_price = float(data_ts['3. low'])
        close_price = float(data_ts['4. close'])
        volume = float(data_ts['5. volume'])
        symbol = symbol
#for date, data_ti in data_ti.items():
  #ATR_alpha = float(data_ti["ATR"])

   # Create a DataFrame for the symbol's data
        combined_data = combined_data.append({
            'Date': date,
            'Open': open_price,
            'High': high_price,
            'Low': low_price,
            'Close': close_price,
            'Volume': volume,
            #'ATR_alpha': ATR_alpha,
            'Symbol': symbol}, ignore_index=True)



In [ ]:
combined_data['Date'] = pd.to_datetime(combined_data['Date'])

combined_data['ATR'] = [max(tup) for tup in list(zip(combined_data['High'] - combined_data['Low'],
                                        (combined_data['High'] - combined_data['Close'].shift(1)).abs(),
                                        (combined_data['Low']  - combined_data['Close'].shift(1)).abs()))]

In [ ]:
#Sort values and assign to "fact" dataframe.  Add 20 day and 55 day moving averages to the dataframe
TCKR = combined_data.groupby('Symbol').apply(lambda x: x.sort_values('Date', ascending= True)).reset_index(drop= True)
TCKR['sma20'] = TCKR.groupby('Symbol').rolling(window = 20)['Close'].mean().reset_index(drop=True)
TCKR['sma55'] = TCKR.groupby('Symbol').rolling(window = 55)['Close'].mean().reset_index(drop=True)
TCKR = TCKR.groupby('Symbol').apply(lambda x: x.sort_values('Date', ascending= False)).reset_index(drop = True)
TCKR = TCKR.sort_index(ascending = False)

TCKR.tail()

In [ ]:
# #Assign signal and trading strategy to each row
TCKR["signal"] = None
for i in range(len(TCKR)):
  if TCKR.iloc[i]["Close"] > TCKR.iloc[i]["sma20"]:
    TCKR.at[i,"signal"] = "Buy as S1"
  elif TCKR.iloc[i]["Low"] < TCKR.iloc[i-10]["sma20"]:
    TCKR.at[i,"signal"] = "Sell as S1 Long"
  elif TCKR.iloc[i]["High"] > TCKR.iloc[i-10]["sma20"]:
    TCKR.at[i,"signal"] = "Sell as S1 Short"

for i in range(len(TCKR)):
  if TCKR.iloc[i]["Close"] > TCKR.iloc[i]["sma55"]:
    TCKR.at[i,"signal"] = "Buy as S2"
  elif TCKR.iloc[i]["Close"] > TCKR.iloc[i-20]["sma20"] and TCKR.iloc[i]["Low"] < TCKR.iloc[i]["sma55"]:
    TCKR.at[i,"signal"] = "Sell as S2 Short"
  elif TCKR.iloc[i]["Close"] > TCKR.iloc[i-20]["sma20"] and TCKR.iloc[i]["High"] < TCKR.iloc[i]["sma55"]:
    TCKR.at[i,"signal"] = "Sell as S2 Long"

TCKR= TCKR.fillna(0)
TCKR = TCKR.sort_index(ascending = True)
TCKR.to_csv("TCKR.csv")

# #Calculate position size and print out information to execute a trade

risk = 0.02
cash = float(input('How much is in account? '))
risk_amount = risk * cash

for x in symbols:
  N = (TCKR.loc[TCKR['Symbol'] == x, 'ATR'].iloc[1])
  signal = (TCKR.loc[TCKR['Symbol'] == x, 'signal'].iloc[1])
  entry_price = (TCKR.loc[TCKR['Symbol'] == x, 'Close'].iloc[1])

  stop_loss = round(((entry_price)- (2*N)), 2)
  position_size= round(risk_amount / stop_loss, 2)
  print(x, {position_size}, f'Stop loss: {stop_loss}, signal = {signal}')

In [ ]:
TCKR.info()

In [ ]:
#S1 filter.  Ignore S1 trades is the previous breakout would have resulted in a winning trade.
for x in symbols:
  lookback20 = (TCKR.loc[TCKR['Symbol'] == x, 'High'].iloc[20])
  lookback40 = (TCKR.loc[TCKR['Symbol'] == x, 'High'].iloc[40])
  stop_loss20 = round(((lookback20)- (2*N)), 2)
  stop_loss40 = round(((lookback40)- (2*N)), 2)

  print(x, f'Lookback 40: {lookback40}, Lookback 20: {lookback20}, Stop loss at 20: {stop_loss20}, Stop loss at 40: {stop_loss40}')
  if float(lookback40) > float(stop_loss40) and float(lookback20) > float(stop_loss40):
    print("You cannot initiate a S1 trade.  Wait for an S2.")
    print("                                                ")
  else:
    print("GO FOR S1 TRADE!")
    print("                              ")